In [1]:
##### Module G6 Challenge: WeatherPy
  ### Part I. Get the Weather Description and Amount of Precipitation for Each City
   ## I.2.a Add the latitudes and longitudes to a list.
   ## I.3.Get the nearest city using the citipy module. 
   ## I.4-5.Perform an API Call with the OpenWeatherMap.
    #    Return: Latitude and Longitude, Max Temp, Percent Humidity, Percent Cloudiness, Wind Speed, Weather description  
    #    Return: Rainfall -use try-except, if it is raining get rainfall # in last three hours. If not raining, add 0 for city
    #    Return: Snow -use try-except, if it is snowing get snowfall # in last three hours. If not snowing, add 0 for city    
   ## I.6. Add collected data to new dataframe
   ## I.7. Save new DataFrame as CSV to be used later in Part II.  weatherPy_challenge.csv
   ## I.8. Git hub 
   ## I.9. ?? Using Pandas: How many cities have recorded rainfall or snowfall

### Part II. Provide inputs for customers to narrow Travel Searches based on Temp and Rain


## I.2.a Add the latitudes and longitudes to a list.
## I.3.Get the nearest city using the citipy module. 
## I.4-5.Perform an API Call with the OpenWeatherMap.
#    Return: Latitude and Longitude, Max Temp, Percent Humidity, Percent Cloudiness, Wind Speed, Weather description  
#    Return: Rainfall -use try-except, if it is raining get rainfall # in last three hours. If not raining, add 0 for city
#    Return: Snow -use try-except, if it is snowing get snowfall # in last three hours. If not snowing, add 0 for city    
## I.6. Add collected data to new dataframe
## I.7. Save new DataFrame as CSV to be used later in Part II.  weatherPy_challenge.csv
## I.8. Git hub 
## I.9. ?? Using Pandas: How many cities have recorded rainfall or snowfall


In [4]:
###   Prepare Environment   
import requests  # needed for web crawls 
import pandas as pd # our Panda friend
import matplotlib.pyplot as plt # matplotlib for plotting
import numpy as np  # numpy for calculations
from citipy import citipy # Wendy Peng's amazing cities library
from datetime import datetime # working with dates
import time  # working with time

##  import our api keys
from config import weather_api_key  ## for weather data
from config import g_key ## for gmaps heatmaps

In [5]:
###  I.2. Generate a new set of 1,500 random latitude and longitude combinations.
#         Using NumPy random number generation with dictated lows, high and size of sample
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)  ## 
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs) ## packs both variables into a tupled zip file
lat_lngs # show me the zip file

In [6]:
##  I.2.a Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs) #write the zip file into a list, so you can work with it

In [7]:
###  I.3.Get the nearest city using the citipy module. 

## Create a list for holding the cities.
cities = []  # empty list

## Identify the nearest city for each latitude and longitude combination.
## The citipy module finds the nearest city to the latitude and longitude pair with a population of 500 or more.

for coordinate in coordinates:  # loop thru coordinates llist of 1500 lats and lngs
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    try:
    # If the city is unique, then we will add it to the cities list.
        if city not in cities:
            cities.append(city)
    except():
        print("General Error... skipping.")   
        # Print the city count to confirm sufficient count.
len(cities)

622

In [8]:
### I.4-5.Perform an API Call with the OpenWeatherMap.
 
 ## I.4.a General a url for querying openweathermap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=058ccb02f765c0268667a6f8eeac8cb8


In [9]:
### I.5 Perform an API Call with the OpenWeatherMap.

##  Prepare empty city list for processing the import

# start with an empty list for weather data
city_data=[]  # empty list

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1  # number of records in crawl
set_count = 1  # # of sets in crawl.  50 cities per set.

# Loop through all the cities in the list.
for i, city in enumerate(cities): # for loop with enumerate. reference index and city var

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50): ## if index divisble by 50 and index> 50, creates a new group
        set_count += 1  # adds one to set count 
        record_count = 1 # resets record count to 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        
        ## Challenge I.5 Weather Description
        city_weather_main=city_weather["weather"][0]["main"]   
        city_weather_desc = city_weather["weather"][0]["description"]
        
        ## Challenge I.5 try - Get inches of rain in last three hours where it has rained.
        try:
            if city_weather["weather"][0]["main"]=="Rain":
                city_rain_inches= city_weather["rain"]["3h"]
            else: 
                city_rain_inches=0
        except:  print (f"failing on the rain. skipping")
             
        ## Challenge I.5 try - except practice.  Now do with snow
        try:
            if city_weather["weather"][0]["main"]=="Snow":
                city_snow_inches= city_weather["snow"]["3h"]
            else: 
                city_snow_inches=0
        except:  print (f"failing on the snow. skipping")
              
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Overall Weather" : city_weather_main,
                          "Weather Description" : city_weather_desc,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Rain Inches (last 3 hrs)" : city_rain_inches,
                          "Snow Inches (last 3 hrs)" : city_snow_inches
                          })
         
                         
# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass  ## general purpose statement to handle all errors and continue

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | saint-philippe
Processing Record 2 of Set 1 | cape town
Processing Record 3 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 4 of Set 1 | ilo
Processing Record 5 of Set 1 | serrai
City not found. Skipping...
Processing Record 6 of Set 1 | flinders
Processing Record 7 of Set 1 | dubai
Processing Record 8 of Set 1 | kapaa
Processing Record 9 of Set 1 | luderitz
Processing Record 10 of Set 1 | coahuayana
Processing Record 11 of Set 1 | hamilton
Processing Record 12 of Set 1 | ribeira grande
Processing Record 13 of Set 1 | albany
Processing Record 14 of Set 1 | shingu
Processing Record 15 of Set 1 | puerto ayora
Processing Record 16 of Set 1 | guerrero negro
Processing Record 17 of Set 1 | bathsheba
failing on the rain. skipping
Processing Record 18 of Set 1 | tezu
Processing Record 19 of Set 1 | punta arenas
Processing Record 20 of Set 1 | te anau
Processing Record 21 of Set 1 | 

City not found. Skipping...
Processing Record 24 of Set 4 | saint-paul
Processing Record 25 of Set 4 | cherskiy
Processing Record 26 of Set 4 | clyde river
Processing Record 27 of Set 4 | viligili
City not found. Skipping...
Processing Record 28 of Set 4 | saquarema
Processing Record 29 of Set 4 | kieta
Processing Record 30 of Set 4 | warqla
City not found. Skipping...
Processing Record 31 of Set 4 | narsaq
Processing Record 32 of Set 4 | kichmengskiy gorodok
Processing Record 33 of Set 4 | sorvag
City not found. Skipping...
Processing Record 34 of Set 4 | pitimbu
Processing Record 35 of Set 4 | cururupu
Processing Record 36 of Set 4 | lebu
Processing Record 37 of Set 4 | araquari
Processing Record 38 of Set 4 | longyearbyen
Processing Record 39 of Set 4 | antofagasta
Processing Record 40 of Set 4 | brae
Processing Record 41 of Set 4 | stepnyak
Processing Record 42 of Set 4 | madison heights
failing on the rain. skipping
Processing Record 43 of Set 4 | kazalinsk
City not found. Skippin

failing on the rain. skipping
Processing Record 5 of Set 8 | sao joao da barra
Processing Record 6 of Set 8 | broome
Processing Record 7 of Set 8 | arrecife
Processing Record 8 of Set 8 | batticaloa
Processing Record 9 of Set 8 | kununurra
Processing Record 10 of Set 8 | mount darwin
Processing Record 11 of Set 8 | omsukchan
Processing Record 12 of Set 8 | warangal
Processing Record 13 of Set 8 | jati
Processing Record 14 of Set 8 | sinnamary
Processing Record 15 of Set 8 | pedernales
Processing Record 16 of Set 8 | banda aceh
Processing Record 17 of Set 8 | tefe
Processing Record 18 of Set 8 | shihezi
Processing Record 19 of Set 8 | lukulu
Processing Record 20 of Set 8 | paoua
Processing Record 21 of Set 8 | vani
Processing Record 22 of Set 8 | japura
Processing Record 23 of Set 8 | jesus carranza
Processing Record 24 of Set 8 | houma
Processing Record 25 of Set 8 | nouadhibou
Processing Record 26 of Set 8 | indiana
Processing Record 27 of Set 8 | broken hill
Processing Record 28 of S

Processing Record 38 of Set 11 | maple creek
Processing Record 39 of Set 11 | tanout
Processing Record 40 of Set 11 | port macquarie
Processing Record 41 of Set 11 | matara
Processing Record 42 of Set 11 | beroroha
Processing Record 43 of Set 11 | guarapari
Processing Record 44 of Set 11 | lebedinyy
Processing Record 45 of Set 11 | mount gambier
Processing Record 46 of Set 11 | aflu
City not found. Skipping...
Processing Record 47 of Set 11 | qabaqcol
Processing Record 48 of Set 11 | mazyr
Processing Record 49 of Set 11 | ulu-telyak
Processing Record 50 of Set 11 | christchurch
Processing Record 1 of Set 12 | cayenne
Processing Record 2 of Set 12 | coihaique
failing on the rain. skipping
Processing Record 3 of Set 12 | kostino
Processing Record 4 of Set 12 | porbandar
Processing Record 5 of Set 12 | gat
Processing Record 6 of Set 12 | waingapu
Processing Record 7 of Set 12 | kralendijk
Processing Record 8 of Set 12 | cairns
failing on the rain. skipping
Processing Record 9 of Set 12 | 

In [10]:
### I.6 Add the collected data to a new dataframe.
city_data_df=pd.DataFrame(city_data)
city_data_df.head(5)
#city_data

,City,Country,Date,Lat,Lng,Overall Weather,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Saint-Philippe,RE,2020-01-14 19:23:20,-21.36,55.77,Rain,light rain,80.60,69,96,6.93,0.38,0.0
1,Cape Town,ZA,2020-01-14 19:23:20,-33.93,18.42,Clouds,few clouds,73.99,94,20,20.80,0.00,0.0
2,Ilo,PE,2020-01-14 19:23:20,-17.64,-71.34,Clouds,scattered clouds,78.80,65,39,10.29,0.00,0.0
3,Flinders,AU,2020-01-14 19:23:21,-34.58,150.86,Clouds,few clouds,66.00,100,17,7.45,0.00,0.0
4,Dubai,AE,2020-01-14 19:19:00,25.26,55.30,Clouds,scattered clouds,66.20,48,40,8.05,0.00,0.0


In [11]:
### I.7  Output the content to a CSV file so we have static data to work with...
output_data_file = "weather_data/weatherPy_challenge.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
## I.8  Sync files with GitHub 

In [60]:
## I.9.a Using Pandas: How many cities have recorded rainfall
city_data_df[city_data_df["Rain Inches (last 3 hrs)"] > 0 ].count()
# Returns 72 Cities with a Rain count

City                        72
Country                     72
Date                        72
Lat                         72
Lng                         72
Overall Weather             72
Weather Description         72
Max Temp                    72
Humidity                    72
Cloudiness                  72
Wind Speed                  72
Rain Inches (last 3 hrs)    72
Snow Inches (last 3 hrs)    72
dtype: int64

In [69]:
## I.9.b Using Pandas: How many cities have recorded snowfall
city_data_df[city_data_df["Snow Inches (last 3 hrs)"] > 0 ].count()
# Returns 23 Cities with a Snow count

City                        23
Country                     23
Date                        23
Lat                         23
Lng                         23
Overall Weather             23
Weather Description         23
Max Temp                    23
Humidity                    23
Cloudiness                  23
Wind Speed                  23
Rain Inches (last 3 hrs)    23
Snow Inches (last 3 hrs)    23
dtype: int64

In [180]:
##  that wraps up Part 1.

In [59]:
### Part II. Provide inputs for customers to narrow Travel Searches based on Temp and Rain - new Jupyter Notebook.


City                        72
Country                     72
Date                        72
Lat                         72
Lng                         72
Overall Weather             72
Weather Description         72
Max Temp                    72
Humidity                    72
Cloudiness                  72
Wind Speed                  72
Rain Inches (last 3 hrs)    72
Snow Inches (last 3 hrs)    72
dtype: int64